In [1]:
import pandas as pd 
import numpy as np
# import cupy, cudf 

# import cupy as np
# import cudf as pd
import time
import pickle
# from cuml.preprocessing import LabelEncoder 
# from cuml.ensemble import RandomForestClassifier
# from cuml.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
# from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
data = pd.read_csv('/home/jude/Downloads/current_customers.csv')
packages = [
    'whole_home_wifi',
    'wifi_security',
    'premium_tech_pro',
    'identity_protection', 
    'family_identity_protection',
    'total_shield',
    'youtube_tv'
]

# Handle NaN values in packages
for p in packages:
    data[p].fillna(False, inplace=True)
    

data['city_state'] = data['city'] + ', ' + data['state']
data['city_state'] = data['city_state'].str.lower()
data['city_state'] = data['city_state'].astype('category')
data['city_state'] = data['city_state'].cat.codes
data['city_state'] = data['city_state'] / data['city_state'].max()

# Handle NaNs in city and state
data['city_state'].fillna(-1, inplace=True)

features = [
    'wired_clients_count',
    'wireless_clients_count',
    'network_speed',
    'city_state'
]
data['network_speed'] = data['network_speed'].str.replace('M', '').astype('float')

feat_df = data[features]
feat_df['city_state'].unique()

wired_clients_median = feat_df['wired_clients_count'].median()
feat_df['wired_clients_count'] = (feat_df['wired_clients_count'] - wired_clients_median) / wired_clients_median 
feat_df['wired_clients_over_median'] = feat_df['wired_clients_count'] > 0
feat_df['wired_clients_count'] = np.abs(feat_df['wired_clients_count'])

wireless_clients_median = feat_df['wireless_clients_count'].median()
feat_df['wireless_clients_count'] = (feat_df['wireless_clients_count'] - wireless_clients_median) / wireless_clients_median 
feat_df['wireless_clients_over_median'] = feat_df['wireless_clients_count'] > 0
feat_df['wireless_clients_count'] = np.abs(feat_df['wireless_clients_count'])

feat_df['network_speed'] = feat_df['network_speed'] / feat_df['network_speed'].max()

# Target 
target = data[packages].sum(axis=1) > 0
x_train, x_test, y_train, y_test = train_test_split(feat_df, target, test_size=0.2)

x_train

print(f'Training on {len(x_train)}...')
start = time.time()
# model = RandomForestClassifier()
model = LogisticRegression()

model.fit(x_train, y_train)
end = time.time()
print(f'Time to train: {end - start}')

print('Inference...')
start = time.time()
y_test = y_test.to_numpy()
y_pred = model.predict(x_test)#.to_numpy()
end = time.time()
print(f'Time to inference: {end - start}')

acc = accuracy_score(y_test, y_pred)
print(f'Accuracy: {acc}')

In [ ]:
info = {
    'model': model,
    'wireless_clients_median': wireless_clients_median,
    'wired_clients_median': wired_clients_median
}

with open('model.pkl', 'wb') as f:
    # pickle.dump(model, f)
    pickle.dump(info, f)